# Algorithme des *k* plus proches voisins
Le lien vers [openclassroom](https://openclassrooms.com/fr/courses/4011851-initiez-vous-au-machine-learning/4022441-tp-entrainez-le-modele-des-k-plus-proches-voisins-k-nn)

Celui vers le [TP de l'université de Lille donné dans moodle](http://www.grappa.univ-lille3.fr/~ppreux/ensg/miashs/l3-ap/tps/kppv/)

L'objectif de ce TP est de faire reconnaître automatiquement par l'ordinateur des chiffres manuscrits (pour lire des chèques par exemple).

[Autre site](http://www.math.univ-toulouse.fr/~besse/Wikistat/Notebooks/Atelier1-MNIST-python.html)

## 1. Lecture du jeux d'exemples

Avant même d'implanter l'algorithme des *k* plus proches voisins, nous avons besoin d'exemples qui seront traités par l'algorithme. Aussi, commençons par lire un jeu de données.

Il s'agit un dataset très célèbre appelé MNIST. Il est constitué d'un ensemble de 70000 images 28x28 pixels en noir et blanc annoté du chiffre correspondant (entre 0 et 9).

![Un extrait du type d'images que l'on trouve dans le dataset MNIST](extrait MNIST.png)

In [2]:
from data_target import data,target # importation des données d'apprentissage

from random import randint
from PIL import Image

In [15]:
[target.count(i) for i in range(10)] # Donne le nombre de chaque chiffre dans la base

[512, 514, 543, 496, 508, 468, 502, 529, 469, 459]

In [55]:
def AfficherImage(image):
    img = Image.new('L',(28,28))
    pixels=img.load()
    for i in range(28):
        for j in range(28):
            pixels[i,j]=image[j*28+i]
    display(img)
    img.close()

In [56]:
n=randint(0,len(data)-1)
image=data[n] # une image, liste de nombres

s=target[n]
print(s) # le caractère associé à l'image

AfficherImage(image)

3


## 2. Déterminer le plus proche voisin d'un point
### Distance entre deux images
Avant toute chose, il nous faut définir une fonction qui mesure la distance entre deux images. Pour faire les choses les plus simples possibles pour l'instant, définissons une fonction distance qui prend deux images (deux listes de nombres) en paramètre et renvoie la somme des différences de chaque pixel (en valeur absolue).

In [57]:
def distance (a, b):
    somme = 0
    for i in range(784):
        somme += abs(a [i] - b [i])
    return (somme)

In [58]:
image1 = data[569]
image2 = data[65]
distance(image1,image2)

35263

### Le plus proche voisin
Écrire une fonction lePlusProcheVoisin qui prend en paramètre une image (une liste de 784 entiers compris entre 0 et 255) et renvoie l'indice dans data du plus proche voisin.

In [79]:
def lePlusProcheVoisin (x,from_inside=False):
    lePlusPres = 0
    distanceMin = float("inf")
    for i in range(len(data)):
        di = distance (x, data[i])
        if (not from_inside or di != 0) and di < distanceMin:
            lePlusPres = i
            distanceMin = di
    return (lePlusPres)

In [80]:
indice = lePlusProcheVoisin(image,from_inside=True)
AfficherImage(data[indice])

## 3. Déterminer les *k* plus proches voisins d'un point
Écrire une fonction lesKplusProchesVoisins qui prend en paramètre une image et une valeur de k et renvoie la liste des indices dans data des k plus proches voisins.
Quand vous prenez k = 1, cette fonction doit renvoyer le même résultat que la précédente, mis à part le fait que lePlusProcheVoisin renvoie une valeur numérique alors que lesKplusProchesVoisins renvoie une liste d'un élément.

In [70]:
def lesKplusProchesVoisins (x, k):
    
    def lt(e1,e2):
        return e1[1]<=e2[1]
    
    def InsertionTrieeLimitee(e,l,limit):
        if limit>0:
            if l==[]:
                return [e]
            else:
                if lt(e,l[0]):
                    return [e]+l[:min(limit-1,len(l))]
                else:
                    return [l[0]]+InsertionTrieeLimitee(e,l[1:],limit-1)
        else:
            return []
    
    kppv=[]
    for i in range(len(data)):
        kppv=InsertionTrieeLimitee((i,distance(data[i],x)),kppv,k)
    return [e[0] for e in kppv]

In [71]:
Kppv = lesKplusProchesVoisins (image, 5)
for indice in Kppv:
    AfficherImage(data[indice])

## 4. Prédire l'étiquette d'une donnée

Écrire une fonction *predire* qui prend en paramètre une image dans le même format que celles de data et un entier *k* et retourne le chiffre qui est prédit, c'est à dire le chiffre qui est supposé être représenté sur l'iamge.
On décide du chiffre représenté sur l'image en appliquant un choix à la majorité, à savoir le chiffre qui apparaît majoritairement sur les *k* plus proches voisins.

In [72]:
def predire (l,k):
    Kppv = lesKplusProchesVoisins(l,k)
    decomptes = [0]*10
    for indice in Kppv:
        decomptes[target[indice]] += 1
    plusGrandDecompte = decomptes [0]
    for i in range (1,10):
        if decomptes [i] > plusGrandDecompte:
                plusGrandDecompte = decomptes [i]
                indice = i
    return (indice)

In [91]:
img=Image.open('Scan_image.pgm') # Image enregistrée en brut sous gimp
image=list(img.getdata())
display(img)
img.close()

Kppv = lesKplusProchesVoisins (image, 5)
for indice in Kppv:
    AfficherImage(data[indice])

predire(image,10)

1

## 5. Optimisation

In [86]:
from data_target_test import data_test,target_test

def taux_erreur(k):
    '''
    calcule le taux d'erreur avec la valeur k pour les k plus proches voisins
    '''
    t=0
    n=100 #len(target_test)
    for i in range(n):
        if predire(data_test[i],k)!=target_test[i]:
            t+=1
    return t/n

def optimisation(n):
    '''
    détermine quelle valeur de k donne la meilleure prédiction, avec k entre 2 et n
    '''
    liste_taux=[taux_erreur(k) for k in range(2,n+1)]
    return liste_taux.index(min(liste_taux))+2

In [87]:
taux_erreur(5)

0.19

In [92]:
optimisation(6)

2